In [18]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import LogFormatterSciNotation
from matplotlib.ticker import PercentFormatter
from matplotlib.ticker import LogFormatterMathtext
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
import os
import sys
from datetime import timedelta, date
import csv
mpl.rcParams['figure.dpi'] = 300
import matplotlib.ticker as ticker

In [19]:
mpl.rcParams['figure.dpi'] = 175

In [3]:
okabe_ito = ["#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
sns.set_context('paper')
sns.set_palette(sns.color_palette(okabe_ito))

In [4]:
#https://stackoverflow.com/questions/579310/formatting-long-numbers-as-strings-in-python/49955617#49955617
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])
def make_cdf(series):
    return series.value_counts(normalize=True).sort_index().cumsum()
def better_general_format(num):
    if num >= 2**7:
        return r'$2^{%s}$' % format(np.log(num)/np.log(2), 'g')
    elif num <= -(2**7):
        num = -num
        return r'-$2^{%s}$' % format(np.log(num)/np.log(2), 'g')
    num = format(num, 'g')
    return num

In [5]:
conn = sqlite3.connect("../agg-data/agg_fcc.db")
df = pd.read_sql_query("select * from agg_httpgetmt;", conn)
conn.close()

In [20]:
conn_unit_profile = sqlite3.connect("../agg-data/unit-profile.db")
df_tech = pd.read_sql_query('select * from "unit-profile"', conn_unit_profile)

#fix AT&T IPBB
df_tech['technology'] = np.where(df_tech.isp == 'AT&T IPBB', 'DSL', df_tech.technology)
df_tech['technology'] = np.where(df_tech.technology == 'IPBB', 'DSL', df_tech.technology)
df_tech['technology'] = np.where(df_tech.technology == 'UVERSE', 'DSL', df_tech.technology)

def isp_names(df, column):
    df[column] = np.where(df[column].str.startswith("Verizon"), 'Verizon', df[column])
    df[column] = np.where(df[column] == "TimeWarner", 'Time Warner Cable', df[column])

    df[column] = np.where(df[column].str.startswith("Frontier"), 'Frontier', df[column])
    df[column] = np.where(df[column] == "Qwest", 'CenturyLink', df[column])
    df[column] = np.where(df[column].str.startswith("Cincinnati Bell"), 'Cincinnati Bell', df[column])
    df[column] = np.where(df[column].str.startswith("AT&T"), 'AT&T', df[column])
    df[column] = np.where(df[column] == "TWC", 'Time Warner Cable', df[column])
    
isp_names(df_tech, 'isp')

df_satellite_isps = df_tech[(df_tech.year == '2017') & (df_tech.isp.isin(['Hughes', 'Wildblue/ViaSat']))]
df_satellite_isps = df_satellite_isps[~df_satellite_isps.unit_id.isin(df_tech[df_tech.year.isin(['2019', '2018'])].unit_id.unique())]
usable_sat_units = df_satellite_isps.unit_id.unique().tolist()
usable_sat_units.remove(1273)
usable_sat_units.remove(801252)
usable_sat_units.remove(13236)
usable_sat_units.remove(999051)
usable_sat_units.remove(216686)

df_att = df_tech[(df_tech.year == '2018') & (df_tech.isp.isin(['AT&T']))]
df_att = df_att[~df_att.unit_id.isin(df_tech[df_tech.year == '2019'].unit_id.unique())]
att_probes_carry_fwd = df_att.unit_id.unique().tolist()
att_probes_carry_fwd.remove(507)
att_probes_carry_fwd.remove(5655)
att_probes_carry_fwd.remove(7460)
att_probes_carry_fwd.remove(27738)
att_probes_carry_fwd.remove(216054)
att_probes_carry_fwd.remove(216146)
att_probes_carry_fwd.remove(805166)
att_probes_carry_fwd.remove(805490)
att_probes_carry_fwd.remove(805630)
att_probes_carry_fwd.remove(805782)
att_probes_carry_fwd.remove(811954)
att_probes_carry_fwd.remove(941780)
att_probes_carry_fwd.remove(994195)
att_probes_carry_fwd.remove(994419)
att_probes_carry_fwd.remove(999719)

#Hawaiian Telcom -- remove, bought by cincinnati bell
df_hitel = df_tech[(df_tech.year == '2017') & (df_tech.isp.isin(['Hawaiian Telcom']))]
df_hitel = df_hitel[~df_hitel.unit_id.isin(df_tech[df_tech.year.isin(['2019', '2018'])].unit_id.unique())]
hit_probes_carry_fwd = df_hitel.unit_id.unique().tolist()
hit_probes_carry_fwd.remove(995125)
hit_probes_carry_fwd.remove(995121)
hit_probes_carry_fwd = []

df_tech_for_merge = pd.DataFrame()
for year in ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']:
    #print(year)
    if year == '2012':
        df_tmp = df_tech[df_tech.year == '2012']
        df_tmp_for_append = df_tmp[df_tmp.month == '09']
        list_2012_newer_units = df_tmp_for_append.unit_id.to_list()
        list_2012_older_units = df_tmp[df_tmp.month == '04'].unit_id.to_list()
        for unit_id in list_2012_older_units:
            if not unit_id in list_2012_newer_units:
                df_tmp_for_append = pd.concat([df_tmp_for_append, df_tmp[(df_tmp.month == '04') & (df_tmp.unit_id == unit_id)]], ignore_index=True)
    elif year == '2013':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2014':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2015':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2016':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2017':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2018':
        df_tmp_for_append = df_tech[(df_tech.year == '2017') & (df_tech.unit_id.isin(usable_sat_units + hit_probes_carry_fwd))]
        df_tmp_for_append['year'] = '2018'
        df_tmp_for_append = pd.concat([df_tmp_for_append, df_tech[df_tech.year == year]], ignore_index=True)
    elif year == '2019':
        df_tmp_for_append = df_tech[(df_tech.year == '2017') & (df_tech.unit_id.isin(usable_sat_units + hit_probes_carry_fwd + att_probes_carry_fwd))]
        df_tmp_for_append['year'] = '2019'
        df_tmp_for_append = pd.concat([df_tmp_for_append, df_tech[df_tech.year == year]], ignore_index=True)
        
    df_tech_for_merge = pd.concat([df_tech_for_merge, df_tmp_for_append], ignore_index=True)

/u/home/fries/.local/lib/python3.6/site-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/u/home/fries/.local/lib/python3.6/site-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
df_tech_for_merge = df_tech_for_merge[~df_tech_for_merge.download.isna()]

In [8]:
df_tech_for_merge[~df_tech_for_merge.download.str.startswith('[')].download.unique()

array(['15', '35', '3', '105', '30', '6', '10', '100', '1.5', '0.768',
       '150', '50', '12', '25', '20', '18', '0.512', '5', '24', '22',
       '75', 'REMOVE', '28', '40', '7', '8', '4', '1', '2', '9', '0.288',
       '0.0', '30.0', '15.0', '7.0', '50.0', '6.0', '20.0', '3.0', '10.0',
       '8.0', '35.0', '1.0', '25.0', '16.0', '24.0', '12.0', '22.0',
       '40.0', '18.0', '28.0', '5.0', '2.0', '9.0', '4.0', '100.0', '45',
       'REMOVE - DATA', '60', '90', '101', 'UNSURE', '300', 'remove',
       'MISC', 'Misc', '16', '200', '14', '11', '70', '120', '250'],
      dtype=object)

In [23]:
df_tech_for_merge[df_tech_for_merge.download.str.startswith('[')].download.unique()

array(['[1.1 - 3.0]', '[3.1 - 7.0]', '[0.5 - 1.0]', '[7.1 - 15.0]'],
      dtype=object)

In [24]:
df_tech_for_merge[df_tech_for_merge.upload.str.startswith('[')].upload.unique()

array(['[0.384 - 0.768]', '[0.768]', '[0.384]', '[1.0]'], dtype=object)

In [25]:
df_tech_for_merge['download'] = np.where(df_tech_for_merge['download'] == '[1.1 - 3.0]', '2.05', df_tech_for_merge['download'])
df_tech_for_merge['download'] = np.where(df_tech_for_merge['download'] == '[0.5 - 1.0]', '0.75', df_tech_for_merge['download'])
df_tech_for_merge['download'] = np.where(df_tech_for_merge['download'] == '[7.1 - 15.0]', '11.05', df_tech_for_merge['download'])
df_tech_for_merge['download'] = np.where(df_tech_for_merge['download'] == '[3.1 - 7.0]', '5.05', df_tech_for_merge['download'])



df_tech_for_merge['upload'] = np.where(df_tech_for_merge['upload'] == '[0.384 - 0.768]', '0.576', df_tech_for_merge['upload'])
df_tech_for_merge['upload'] = np.where(df_tech_for_merge['upload'] == '[0.384]', '0.384', df_tech_for_merge['upload'])
df_tech_for_merge['upload'] = np.where(df_tech_for_merge['upload'] == '[1.0]', '1.0', df_tech_for_merge['upload'])
df_tech_for_merge['upload'] = np.where(df_tech_for_merge['upload'] == '[0.768]', '0.768', df_tech_for_merge['upload'])

In [26]:
df_tech_for_merge['download'] = pd.to_numeric(df_tech_for_merge['download'], errors='coerce')
df_tech_for_merge['upload'] = pd.to_numeric(df_tech_for_merge['upload'], errors='coerce')

In [28]:
df_tech_for_merge.dropna(axis=0, inplace=True)

In [30]:
df_tech_for_merge.groupby('year').agg({'download':'median', 'upload':'median'}).reset_index()

,year,download,upload
0,2012,15.0,2.0
1,2013,18.0,2.0
2,2014,20.0,4.0
3,2015,25.0,4.0
4,2016,25.0,4.0
5,2017,45.0,5.0
6,2018,30.0,3.0
7,2019,50.0,5.0


In [17]:
df_tech_for_merge.groupby('year').agg({'download':'max', 'upload':'max'}).reset_index()

,year,download,upload
0,2012,150.0,65.0
1,2013,300.0,65.0
2,2014,300.0,300.0
3,2015,300.0,300.0
4,2016,300.0,75.0
5,2017,300.0,150.0
6,2018,300.0,150.0
7,2019,300.0,150.0


In [9]:
df = df[df.med_bytes_sec > 0.]


df = df[df.sum_suc > 0.]



df['dtime'] = pd.to_datetime(df['dtime'])
df['year'] = df['dtime'].dt.strftime('%Y')




df['med_bytes_sec'] = df['med_bytes_sec'] * 8 / (1000*1000)


In [10]:
df_delta = pd.merge(df_tech_for_merge, df.groupby(['unit_id', 'year'], as_index=False).agg({'med_bytes_sec':'median'}), how='inner', on=['unit_id', 'year'])



In [11]:
df_delta['med_bytes_sec_delta'] = df_delta.med_bytes_sec - df_delta.download


In [58]:
for year in ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']:
    df_tmp = df_delta[df_delta.year == year]["med_bytes_sec_delta"]
    if len(df_tmp) > 0:
        print(year)
        print('')
        tmp_cdf = make_cdf(df_tmp)
        try:
            print(tmp_cdf[np.min(np.abs(tmp_cdf.index))])#[np.amin(make_cdf(df_tmp))])
        except:
            print(tmp_cdf[-np.min(np.abs(tmp_cdf.index))])
        neg_index = tmp_cdf.index[tmp_cdf.index + 8 < 0]
        print(tmp_cdf[np.max(neg_index)])
        print('')
        print(df_tmp.describe(percentiles=[.33, .95, .99, .75, .3497, .3660, .2899, .2537, .3586, .3540], include='all'))
        print('')
        print('')

2012

0.3497897897897956
0.026546546546546534

count    8325.000000
mean        1.949182
std         6.881816
min      -118.497724
25.4%      -0.234283
29.0%      -0.149192
33%        -0.043038
35.0%       0.000063
35.4%       0.008734
35.9%       0.019950
36.6%       0.045505
50%         1.439680
75%         4.690462
95%         8.963850
99%        15.852072
max       139.807880
Name: med_bytes_sec_delta, dtype: float64


2013

0.36605812220565853
0.06482861400894237

count    5368.000000
mean        0.371681
std         9.044198
min      -125.580882
25.4%      -0.379104
29.0%      -0.245182
33%        -0.135820
35.0%      -0.046610
35.4%      -0.034203
35.9%      -0.020414
36.6%       0.000118
50%         0.983103
75%         3.561656
95%         8.696081
99%        14.865503
max        68.659092
Name: med_bytes_sec_delta, dtype: float64


2014

0.2900394123969958
0.10569688283769337

count    5582.000000
mean       -0.131348
std        11.950204
min       -93.571412
25.4%      -0.20